In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import itertools as it
import statistics
from lightfm.datasets import fetch_movielens
import syft as sy #syft==0.2.9

GPUへ切り替え，シード固定

In [2]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)
torch.manual_seed(32)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

データセットにはmovielens 100kを使用
train,testにはユーザの映画閲覧履歴
moviesには各映画のジャンルに関する情報

In [3]:
movielens = fetch_movielens()
train = movielens['train']
test = movielens['test']
train = pd.DataFrame.sparse.from_spmatrix(train)
test = pd.DataFrame.sparse.from_spmatrix(test)

In [4]:
import codecs
with codecs.open('/data/bpr/paper/small/data/ml-100k/u.item', 'r', 'utf-8', errors='ignore') as f:
    movies = pd.read_table(f, delimiter='|', header=None).iloc[:, 5:]
    movies.rename(columns={0: 'item', 1: 'item_title'}, inplace=True)
movie_data = movies.values

In [8]:
n_user, n_item = train.shape
n_genre = len(movies.columns)
item_array = np.array(range(n_item))
user_array = np.array(range(n_user))

連合学習のworkerを定義

In [10]:
workers=[]
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
for i in range(n_user):
  workers.append(sy.VirtualWorker(hook, id=f"group{i+1}"))  # <-- NEW:

全アイテムの組み合わせのジャンルに基づく類似度を算出

In [11]:
def genre_corr(i,j):
    num = movie_data[i]+movie_data[j]
    sim = np.count_nonzero(num==2)/np.count_nonzero(num>0)
    return sim

In [12]:
corr = [[genre_corr(i,j) for j in range(n_item)] for i in range(n_item)]

In [13]:
corr[0][3] #アイテム0とアイテム3のジャンル類似度が0.2

0.2

corrを基に学習データを作成

In [87]:
def sampling(pos,neg,corr):
    while True:
        sample_pos = np.random.choice(pos, 1)[0]
        sample_neg = np.random.choice(neg, 1)[0]
        if 0<corr[sample_pos][sample_neg]<=0.5:
            break
    return (sample_pos, sample_neg)

In [111]:
mem=[]
pair_wise_items = []
for user in range(n_user):
    print(user)
    pos_items = train.columns[train.iloc[user]>0] 
    neg_items = np.setxor1d(item_array, pos_items)
    pair_wise_item = [sampling(pos_items, neg_items, corr) for i in range(2**10)]
    pair_wise_items.append(pair_wise_item)
    positive_item = [a[0] for a in pair_wise_item] 
    negative_item = [a[1] for a in pair_wise_item] 
    positive_item, negative_item = Variable(torch.FloatTensor(positive_item)).long(), Variable(torch.FloatTensor(negative_item)).long()
    mem.append(sy.BaseDataset(torch.tensor(positive_item), torch.tensor(negative_item)).send(workers[user]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [112]:
federated_train_dataset = sy.FederatedDataset(mem)
federated_train_loader = sy.FederatedDataLoader(federated_train_dataset, batch_size=100)

In [113]:
class Model(nn.Module):
    def __init__(self, item, k=10):
        super().__init__()
        self.item_emb = nn.Embedding(item, k,sparse=True)
        self.user_emb = nn.Embedding(1, k ,sparse=True)
    def forward(self,item,d):
        item = self.item_emb(item)
        user = self.user_emb(d)
        return torch.sum(item * user, 1)

In [114]:
model=Model(n_item,k=10)
useremb = torch.randn(n_user, 10)
model.train()

Model(
  (item_emb): Embedding(1682, 10, sparse=True)
  (user_emb): Embedding(1, 10, sparse=True)
)

多様度測定のための関数

In [ ]:
def com(n, p_list):
    rec_list = np.array(range(20))
    com_item = it.combinations(rec_list, n)
    all_pp = 0
    for x in com_item:
        all_p = 1
        pair = np.asarray(x)
        unpair = np.setdiff1d(rec_list, pair)
        for p in pair:
            all_p = all_p * p_list[p]
        for unp in unpair:
            all_p = all_p * (1 - p_list[unp])
        all_pp = all_pp + all_p
    return all_pp

def f(x, pl, p_more_than_0):
    return com(x, pl) / p_more_than_0

def sigma(func, frm, to, pl, p0):
    result = 0;
    for i in range(frm, to+1):
        result += func(i, pl, p0)
    if result >=1:
        result = 0.9999999999999
    else:
        result = round(result, 13) 
    #print("result:{0}".format(result))
    return result
def for_div(useremb,itememb,n,user):
    pscore = (itememb * useremb[user]).sum(axis=1)
    idx = np.argsort(pscore)[::-1]
    rec_item = idx[:n]
    all_genre = np.zeros(n_genre, dtype=np.int)
    all_nonslt = 1
    p_list = []
    for item in rec_item:
        genre = movies.iloc[item].values
        p = np.count_nonzero(genre>0)/len(genre)
        nonslt = 1 - p
        p_list.append(p)
        all_genre = all_genre + genre
        all_nonslt = all_nonslt * nonslt

    n_noncov = np.count_nonzero(all_genre < 1)
    cov = (all_nonslt ** (1/n_genre)) ** n_noncov

    p_more_than_0 = 1 - all_nonslt
    nonred = 1
    n_cov = np.count_nonzero(all_genre > 0)
    for i_genre in all_genre[all_genre>0]:
        p_genre = (1 - sigma(f, 1, i_genre-1, p_list, p_more_than_0)) ** (1/n_cov)
        nonred = nonred * p_genre
        #p_genre = 1 - ((sigma(f, 1, n_genre-1) * p_more_than_0) / p_more_than_0) 
    div = cov * nonred
    return div

類似度測定のための関数

In [ ]:
def for_acc(useremb,itememb,n,test,user):
    pscore = (itememb * useremb[user]).sum(axis=1)
    idx = np.argsort(pscore)[::-1]
    rec_item = idx[:n]
    return len(np.intersect1d(rec_item, test))/n

In [ ]:
def eval(test,useremb,itememb,corr):
    rec_scale = 20
    useremb = useremb.to("cpu").detach().numpy().copy()
    itememb = itememb.to('cpu').detach().numpy().copy()
    acc = [for_acc(useremb,itememb,scale,i) for i in user_lst]
    div = joblib.Parallel(n_jobs=70, verbose=1)(joblib.delayed(for_div)(useremb,itememb,scale,test,i) for i in user_lst)
    accuracy = statistics.mean(acc)
    diversity = statistics.mean(div)
    return accuracy,diversity

損失関数定義

In [115]:
def bpr_loss(x): 
  return -logsigmoid(x[0] - x[1]).mean()
optimizer = optim.SGD(params=model.parameters(),lr=0.1)
def logsigmoid(x):
    x = 1 / (1 + torch.exp(-x))
    return torch.log(x)

In [116]:
for q in range(10):
  print(q)
  for u, (pos,neg) in enumerate(federated_train_loader):
    worker = pos.location
    pos, neg = pos.to(device), neg.to(device)
    useremb.to(device)
    d = torch.tensor(0)
    d = d.send(worker)
    u = u % 943
    model.user_emb.weight.data =  useremb[u].unsqueeze(dim=0)
    model.send(worker)
    predict = [model(pos,d),model(neg,d)] 
    loss = bpr_loss(predict)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.get()
    useremb[u] = model.user_emb(torch.tensor(0)).unsqueeze(dim=0)
  eval(test,useremb,model.item_emb(m),corr)

0


KeyboardInterrupt: 